### Extracao e Limpeza

In [1]:
import pandas as pd
from os import listdir, path

In [2]:
directory = 'data_raw'

In [3]:
cols = ['ANO', 'MES', 'EMPRESA', 'ORIGEM', 'DESTINO', 'TARIFA', 'ASSENTOS']

In [4]:
dfs = []
for filename in listdir(directory):
    f = path.join(directory, filename)
    dfs.append(pd.read_csv(str(f), sep=';', encoding='latin-1', names=cols, skiprows=1, low_memory=False))

In [5]:
df = pd.concat(dfs)

In [6]:
df['TARIFA'] = df['TARIFA'].str.replace(',', '.')
df['TARIFA'] = df['TARIFA'].astype(float)

In [7]:
df

,ANO,MES,EMPRESA,ORIGEM,DESTINO,TARIFA,ASSENTOS
0,2021,1,AZU,SBAC,SBBR,293.2,1
1,2021,1,AZU,SBAC,SBBR,340.9,6
2,2021,1,AZU,SBAC,SBBR,367.3,1
3,2021,1,AZU,SBAC,SBBR,435.9,1
4,2021,1,AZU,SBAC,SBBR,452.9,1
...,...,...,...,...,...,...,...
310582,2021,6,TAM,SNTF,SBSP,526.9,2
310583,2021,6,TAM,SNTF,SBSP,548.9,1
310584,2021,6,TAM,SNTF,SBSP,626.9,1
310585,2021,6,TAM,SNTF,SBSP,671.9,1


In [8]:
df_aerodromos = pd.read_csv('AerodromosPublicos.csv', sep=';', encoding='latin-1', low_memory=False)

In [9]:
drop_cols = ['CIAD', 'Nome', 'Município Servido', 'UF', 'UF Servido', 'LATGEOPOINT', 'LONGEOPOINT', 'Latitude', 'Longitude', 
             'Altitude', 'Operação Diurna', 'Operação Noturna', 'Designação 1', 'Superfície 1', 'Superfície 1', 'Designação 2',
            'Resistência 1', 'Resistência 2', 'Superfície 2', 'Situação', 'Validade do Registro', 'Portaria de Registro', 'Link Portaria',
            'Largura 1', 'Largura 2', 'Comprimento 1', 'Comprimento 2']

In [10]:
df_aerodromos.drop(drop_cols, axis=1, inplace=True)

In [11]:
df_aerodromos.rename(columns={'Código OACI': 'oaci', 'Município': 'cidade'}, inplace=True)

In [12]:
df_aerodromos.set_index('oaci', inplace=True)

In [13]:
df_aerodromos

,cidade
oaci,
SBRB,RIO BRANCO
SNGA,GUARAPARI
SBIZ,IMPERATRIZ
SBPC,POÇOS DE CALDAS
SSCD,CHAPADÃO DO SUL
...,...
SDAG,ANGRA DOS REIS
SWBR,BORBA
SBMO,MACEIÓ


### Join do DF principal com o DF de aerodromos

In [14]:
df = df.merge(df_aerodromos, left_on='ORIGEM', right_on='oaci')
df = df.merge(df_aerodromos, left_on='DESTINO', right_on='oaci')

In [15]:
df.drop(['ORIGEM', 'DESTINO'], axis=1, inplace=True)
df.rename(columns={'cidade_x': 'ORIGEM', 'cidade_y': 'DESTINO'}, inplace=True)

In [16]:
df

,ANO,MES,EMPRESA,TARIFA,ASSENTOS,ORIGEM,DESTINO
0,2021,1,AZU,293.2,1,ARACATI,BRASÍLIA
1,2021,1,AZU,340.9,6,ARACATI,BRASÍLIA
2,2021,1,AZU,367.3,1,ARACATI,BRASÍLIA
3,2021,1,AZU,435.9,1,ARACATI,BRASÍLIA
4,2021,1,AZU,452.9,1,ARACATI,BRASÍLIA
...,...,...,...,...,...,...,...
1883690,2021,3,AZU,55.0,1,SANTARÉM,MONTE ALEGRE
1883691,2021,3,AZU,98.0,2,SANTARÉM,MONTE ALEGRE
1883692,2021,4,AZU,153.0,2,SANTARÉM,MONTE ALEGRE
1883693,2021,1,AZU,55.0,30,SÃO PAULO,ITANHAÉM


In [17]:
df.to_csv(path.join('data_transformed', 'df_main.csv'), index=False)